In [1]:
using POMDPs, POMDPTools, QuickPOMDPs, MCTS, DiscreteValueIteration, POMDPSimulators, POMDPModels, POMDPPolicies
using Distributions, Combinatorics, StaticArrays, D3Trees, Random
using FileIO, JLD2, TickTock

In [2]:
action=[(0,0), (0,1), (1,1)]

Number_level = 4;
#Transition_matrix = [0.86 0.14 0.0 0.0; 0.0 0.83 0.17 0.0;0.0 0.0 0.89 0.11;1.0 0.0 0.0 0.0];
fullname = "tm10";
# generate_transition_matrix(Number_level, fullname) 
global Transition_matrix = 
            [0.8571 0.1429 0 0;
        0 0.8571 0.1429 0;
        0 0 0.8 0.2
      0 0 0 1];
Transition_matrix=Transition_matrix./[sum(Transition_matrix[i,:]) for i in 1:4]
failure_penalty = -1000.0;
maintenance_penalty = -200.0;
setup_cost = -800.0;
normal_operation = 0.0;

In [6]:
function reward(s, a,s1=10, rng=23)       #s is a vector of number units at each level and a is the number of units we will repair
       if s==Number_level
            r=failure_penalty +  setup_cost/NumberUnits  + normal_operation  ;
        return r
       end
        r=0
     if a==1
        if s1>=s
            crd = Categorical(Transition_matrix[s,:]);
            s = rand(crd);
            r = normal_operation ;
         return r
        else
            return 0.0
        end
        end
    if a==2
         if s1>=s
            crd = Categorical(Transition_matrix[s,:]);
            s = rand(crd);
            r = normal_operation +  setup_cost/NumberUnits;
         return r
        else
            return 0.0
        end
        end
    if a==3
        s1=deepcopy(s)
        s=1
        r = maintenance_penalty +  setup_cost/NumberUnits
         crd = Categorical(Transition_matrix[s,:]);
            s = rand(crd);
            r+= normal_operation  ;
#            r*=(s1/10);
        return r
    end
    end

reward (generic function with 3 methods)

In [9]:
(Transition_matrix)

4×4 Matrix{Float64}:
 0.8571  0.1429  0.0     0.0
 0.0     0.8571  0.1429  0.0
 0.0     0.0     0.8     0.2
 0.0     0.0     0.0     1.0

In [10]:
T=zeros(Number_level,Number_level,3);


In [11]:
((1:10)./10)

0.1:0.1:1.0

In [12]:
T[:,:,1].=Transition_matrix;

In [13]:
T[:,:,2].=Transition_matrix;

In [14]:
for j in 1:4
        T[j,:,3].=Transition_matrix[1,:];
        end
# for j in 1:10
#         T[j,:,3].=0.0
#     T[j,1,3]=1.0
#         end

In [15]:
T[4,:,1].=Transition_matrix[1,:]
T[4,:,2].=Transition_matrix[1,:]

4-element view(::Array{Float64, 3}, 4, :, 2) with eltype Float64:
 0.8571
 0.1429
 0.0
 0.0

In [19]:
function explore()
    c=[1/3,1/3,1/3]

     crd = Categorical(c);
     return rand(crd)
    end 
function generative(s, a, rng=23)       #s is a vector of number units at each level and a is the number of units we will repair
        if s==Number_level
        s=1
        crd = Categorical(Transition_matrix[s,:]);
            s = rand(crd);
            r=failure_penalty +  setup_cost/NumberUnits + normal_operation  ;
        return (sp=s, r=r)
       end
        r=0
     if a==1
            crd = Categorical(Transition_matrix[s,:]);
            s = rand(crd);
            r = normal_operation ;
         return (sp=s, r=r)
        end
    if a==2
            crd = Categorical(Transition_matrix[s,:]);
            s = rand(crd);
            r = normal_operation +  setup_cost/NumberUnits ;
             return (sp=s, r=r)
        end
    if a==3
        s1=deepcopy(s)
        s=1
        r = maintenance_penalty +  setup_cost/NumberUnits
         crd = Categorical(Transition_matrix[s,:]);
            s = rand(crd);
            r+= normal_operation  ;
#            r*=(s1/10);
       return (sp=s, r=r)
    end
    end

generative (generic function with 2 methods)

In [25]:
function ValueIteration(S, T, γ, p1=0.01, d=100)
    Q=zeros((Number_level,3))
#     print(Q)
    V= rand(Float64, (Number_level))*1000
    diff=0
    i=0
    
    

    
    p=[1 1 1 ]
    while i<=2000
#     while  i<=10000
    for s in 1:4
        Threads.@threads for a in 1:3
 
                Q[s,a]= (reward(s,a)) + γ*( sum(T[s,s1,a]*(V[s1]) for (s1) in 1:4 ))
    end
            V[s]=findmax(Q[s,1:3])[1]
        end
        i+=1
        end
    return Q,V
    end

ValueIteration (generic function with 3 methods)

In [33]:


function decison(Q, s,n=0.2)
   a_final=[]
   if Number_level in s
#         print("hi")
        for i in 1:NumberUnits
            append!(a_final,argmax(a->Q[s[i],a], [2,3]))
            end
        return a_final
    end
    q0=0.0
    a_0=repeat(1:1,NumberUnits);
    for i in 1:NumberUnits
            q0+=Q[s[i],1]
    end
    a_final=repeat(1:1,NumberUnits);
    repair= argmax(i->s[i], 1:NumberUnits)
#     print(repair)
    a_final[repair]=3
    b=1
    for i in 1:NumberUnits
            if i!=repair
            a_final[i]=argmax(a->Q[s[i],a], [2,3])
#             println(s[i])
#             println(a_final[i])

            if a_final[i]==3
                b+=1
            end
            end
    end
        q1=0.0
    for i in 1:NumberUnits
            q1+=Q[s[i],a_final[i]]
    end
#     if b<=NumberUnits*n
#         return a_0
#     end
    if q0>q1
            return a_0
        else
            return a_final
        end
end

decison (generic function with 2 methods)

In [35]:
using DataFrames
df= DataFrame(Units=[],n=[],s=[],m=[],f=[],lambda=[],mean=[],std=[])

Row,Units,n,s,m,f,lambda,mean,std
,Any,Any,Any,Any,Any,Any,Any,Any


In [37]:

cost=[ 
     [0 -200 -200 -1000],
    [0 -400 -200 -1000],
    [0 -600 -200 -1000],
    [0 -800 -200 -1000],
    [0 -1000 -200 -1000],
    [0 -1200 -200 -1000],
    [0 -1400 -200 -1000],
    [0 -1600 -200 -1000]
    ]


for units in [3,5,10,15,20,25,30,40,50,60,70,80,100,125,150,175,200]
    for c in cost
   print(units)  
    global normal_operation,setup_cost,maintenance_penalty,failure_penalty=c
     println([normal_operation,setup_cost,maintenance_penalty,failure_penalty])   
     for lambda in 1:1:1
        global  lamb=lambda
    global NumberUnits=units
     Q,V=ValueIteration(s,T,.95,3000)
         println(isnan(Q[1,1]))
       if sum(isnan(Q[1,1]))>0.0
            continue
        end
tick()
trials=10000;
simsteps = 100;
results=zeros(trials)*0.1
Threads.@threads for h in 1:trials
s=repeat(1:1,NumberUnits)
r1=[]
a1=[]   
r=repeat(1:1,NumberUnits)*1.0
 for k in 1:simsteps
    a =decison(Q,s)
            r2=0.0
#     print(a)
    for i in 1:NumberUnits
#             print(generative(s[i],a[i],234))
         s[i],r[i]=generative(s[i],a[i],234);
          r2+=r[i]
            end 
#     println(r2)
#     println(s)
    s=s;
    append!(a1,a)
    append!(r1,r2)
end
rk=0
for i in 1:simsteps
    rk=0.95*rk+r1[simsteps-i+1]
end
#     println(rk)
    results[h]=rk
#     println(results)
end
println(units)
println(lambda)

println(mean(results))

println(std(results))
pushfirst!(df,[units,normal_operation,setup_cost,maintenance_penalty,failure_penalty,lamb,mean(results),std(results)])
        
tock()
end
    println("-------------------------")
end
end

3[0, -200, -200, -1000]
false


[ Info:  started timer at: 2023-03-24T10:13:18.136


3
1
-1767.9221243510785
572.6439403540659
-------------------------
3[0, -400, -200, -1000]


[ Info:             3.017809s: 3 seconds, 17 milliseconds


false


...

[ Info:          135.3465129s: 2 minutes, 15 seconds, 346 milliseconds


-1600, -200, -1000]
false


[ Info:  started timer at: 2023-03-24T11:54:57.052


200
1
-71579.87049125024
2295.667308404637
-------------------------


[ Info:          133.5162528s: 2 minutes, 13 seconds, 516 milliseconds


In [38]:
df

Row,Units,n,s,m,f,lambda,mean,std
,Any,Any,Any,Any,Any,Any,Any,Any
1,200.0,0.0,-1600.0,-200.0,-1000.0,1.0,-71579.9,2295.67
2,200.0,0.0,-1400.0,-200.0,-1000.0,1.0,-68906.1,2099.74
3,200.0,0.0,-1200.0,-200.0,-1000.0,1.0,-65997.2,1900.78
4,200.0,0.0,-1000.0,-200.0,-1000.0,1.0,-62873.1,1746.02
5,200.0,0.0,-800.0,-200.0,-1000.0,1.0,-59528.8,1619.69
6,200.0,0.0,-600.0,-200.0,-1000.0,1.0,-56123.7,1563.27
7,200.0,0.0,-400.0,-200.0,-1000.0,1.0,-52654.5,1486.85
8,200.0,0.0,-200.0,-200.0,-1000.0,1.0,-49138.5,1474.16
9,175.0,0.0,-1600.0,-200.0,-1000.0,1.0,-64874.4,2347.31


In [39]:
using CSV
CSV.write("valueiteration_MAX_200.csv", df)

"valueiteration_MAX_200.csv"